### Training with dataset from Stack Overflow and Testing on Jira, Stack Overflow, App reviews

In [2]:
import autosklearn.classification
import csv
import json
import math
import numpy as np
import re
import warnings

from collections import defaultdict, OrderedDict
from myconfig import *
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, confusion_matrix, roc_curve, auc
from sklearn.model_selection import StratifiedShuffleSplit

N_GRAM_LENGTH = 10
TOTAL_TYPE = 2
summary = dict()
total_n_gram = 0
total_document = 0
score = dict()
id_n_gram_mapping = dict()
comment_summary = dict()
top_vector = list()
n_grams = dict()
print(project_name)

project_name = "jira"
# negative = -1, neutral = 0, postitive = 1
# weight = log(|D|/sdf) * gtf

def read_raw_data():
    with open(DATA_SOURCE+"/dataset.csv", 'r',encoding='latin1') as csvfile:
        global total_document
        reader = csv.DictReader(csvfile)
        for row in reader:
            if row['project'] == "stackoverflow" or row['project'] == project_name:
                comment_summary[total_document] = dict()
                comment_summary[total_document]['type'] = int(row['oracle'])
                comment_summary[total_document]['comment'] = row['text']
                comment_summary[total_document]['project'] = row['project']
                total_document += 1                

def file_len(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1   
    
#read data from n_gram
def read_n_gram():
    n_gram_id = 0
    total_n_gram = file_len(DATA_SOURCE+"/stackoverflow_n_gram_filter")
    print("total_n_gram",total_n_gram)
    with open(DATA_SOURCE+"/stackoverflow_n_gram_filter") as csvfile:
        reader = csv.reader(csvfile, delimiter='\t', quotechar='|')
        for row in reader:
            words = row[5].strip()
            term = tuple(row[5].strip().split(' '))
            if term not in summary:
                summary[term] = dict()
            summary[term] = {'id':n_gram_id,'len':row[1],'gtf':row[2],'df':row[3],'sdf':row[4], 'term':row[5]}
            weight1 = total_document / int(row[4])
            summary[term]['score'] = math.log10(weight1) * int(row[2])
            score[n_gram_id] =  math.log10(weight1) * int(row[2])
            n_grams[n_gram_id] = term
            n_gram_id += 1

def top_score_vector():
    percent = int(len(score) * 100 / 100)
    top_vector.extend(sorted(score,key=score.get,reverse=True)[:percent])
    print("top_vector",len(top_vector))

def n_gram_split():
    for comment_index in comment_summary:
        comment = comment_summary[comment_index]['comment']
        comment_summary[comment_index]['vector'] = dict()
        comment_post_process = re.sub("\s+"," ",re.sub(r"[^A-Za-z0-9]+"," ",comment.replace("\t"," ").replace("\r\n"," ").lower())).split(" ")
        for i in range(len(comment_post_process)):
            for j in range(i,min(i+N_GRAM_LENGTH+1,len(comment_post_process))):
                if(tuple(comment_post_process[i:j+1]) in summary):
                    if(summary[tuple(comment_post_process[i:j+1])]['id'] in top_vector):
                        if summary[tuple(comment_post_process[i:j+1])]['id'] in comment_summary[comment_index]['vector']:
                            comment_summary[comment_index]['vector'][summary[tuple(comment_post_process[i:j+1])]['id']] += 1
                        else: 
                            comment_summary[comment_index]['vector'][summary[tuple(comment_post_process[i:j+1])]['id']] = 1
                            
def vector_idf():
    for i in comment_summary:
         for v in comment_summary[i]['vector']:
            comment_summary[i]['vector'][v] *= score[v]

def cal_coverage():
    empty_vector = 0
    total_vector = len(comment_summary)
    for i in comment_summary:
        if not comment_summary[i]['vector']:
            empty_vector += 1
            print(comment_summary[i],comment_summary[i]['vector'])
    print("coverage ratio:",(total_vector - empty_vector)/total_vector * 100)

def cal_coverage_freq():
    empty_vector = 0
    total_vector = len(comment_summary)
    n_gram_per_comment = defaultdict(lambda:0)
    for i in comment_summary:
        sum = 0
        for j in comment_summary[i]['vector']:
            sum += 1
        n_gram_per_comment[sum] += 1
    print(OrderedDict(sorted(n_gram_per_comment.items(), key=lambda t: t[0])))
    
read_raw_data()
read_n_gram()
top_score_vector()
n_gram_split()
print("finish")
#vector_idf()
cal_coverage()
cal_coverage_freq()

/Users/rungrojmaipradit/anaconda3/lib/python3.7/site-packages/pyparsing.py:2725: FutureWarning: Possible set intersection at position 3
  self.re = re.compile( self.reString )
/Users/rungrojmaipradit/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


stackoverflow
total_n_gram 1383
top_vector 1383
finish
{'type': 0, 'comment': 'gotcha ', 'project': 'stackoverflow', 'vector': {}} {}
{'type': 0, 'comment': 'trust uri ', 'project': 'stackoverflow', 'vector': {}} {}
{'type': 0, 'comment': 'regex explained ', 'project': 'stackoverflow', 'vector': {}} {}
{'type': 0, 'comment': 'short answer ', 'project': 'stackoverflow', 'vector': {}} {}
{'type': 0, 'comment': 'example 35 degrees web address ', 'project': 'stackoverflow', 'vector': {}} {}
{'type': 0, 'comment': 'insertandretriveblobdata ', 'project': 'stackoverflow', 'vector': {}} {}
{'type': 0, 'comment': 'url web address ', 'project': 'stackoverflow', 'vector': {}} {}
{'type': 0, 'comment': 'wildfly developer guide ', 'project': 'stackoverflow', 'vector': {}} {}
{'type': 0, 'comment': 'check textsamplerdemo ', 'project': 'stackoverflow', 'vector': {}} {}
{'type': 0, 'comment': 'ear dependencies ', 'project': 'stackoverflow', 'vector': {}} {}
{'type': 0, 'comment': 'plese guide ', 'proj

In [5]:
# currently we have top_score_vector and comment&vector
# create x and y -> x contains vector and y contains result
# X = []
# y = []
X_train,y_train,X_test,y_test = [],[],[],[]
print("test")
for comment_index in comment_summary:
    vector = [0] * len(top_vector)
    for vector_index in comment_summary[comment_index]['vector']:
        vector[top_vector.index(vector_index)] = comment_summary[comment_index]['vector'][vector_index]
    if comment_summary[comment_index]['project'] == 'stackoverflow':
        X_train.append(vector)
        y_train.append(comment_summary[comment_index]['type'])
    else:
        X_test.append(vector)
        y_test.append(comment_summary[comment_index]['type'])        
print(len(X_train),len(X_test))
print("finish")

test
1500 926
finish


In [13]:
warnings.filterwarnings("ignore")

X_train,y_train,X_test,y_test = np.asarray(X_train),np.asarray(y_train),np.asarray(X_test),np.asarray(y_test)
automl = autosklearn.classification.AutoSklearnClassifier(ml_memory_limit=1024*64,time_left_for_this_task=90*60,per_run_time_limit=15*60)
automl.fit(X_train.copy(), y_train.copy(),metric=autosklearn.metrics.f1_weighted)
automl.refit(X_train.copy(), y_train.copy())
y_hat = automl.predict(X_test)
fpr, tpr, thresholds = roc_curve(y_test, y_hat, pos_label=1)
log = automl.cv_results_

print("Classification report", classification_report(y_test, y_hat))
print("Confusion matrix", confusion_matrix(y_test, y_hat))
print("AUC",auc(fpr,tpr))
print("number of process",len(log['rank_test_scores']))
print("status",log['status'])
print("mean_test_score",log['mean_test_score'])

for i in np.where(log['rank_test_scores']==1)[0] :
    print(log['params'][i])     

for i in range(len(y_test)):
    if y_test[i] != y_hat[i]:
        print("comment index:",len(y_train)+i,"y_test:",y_test[i],"y_hat",y_hat[i])

[WARNING] [2019-01-30 15:36:02,224:EnsembleBuilder(1):5f957d92b05f37228f13aa90de9fbe78] No models better than random - using Dummy Score!
[WARNING] [2019-01-30 15:36:02,238:EnsembleBuilder(1):5f957d92b05f37228f13aa90de9fbe78] No models better than random - using Dummy Score!
[WARNING] [2019-01-30 15:36:04,244:EnsembleBuilder(1):5f957d92b05f37228f13aa90de9fbe78] No models better than random - using Dummy Score!
[WARNING] [2019-01-30 15:38:15,336:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2019-01-30 15:38:15,336:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
Classification report              precision    recall  f1-score   support

         -1       0.99      0.16      0.28       636
          0       0.00      0.00      0.00         0
          1       0.76      0.11      0.19       290

avg / total       0.92      0.14      0.

comment index: 1947 y_test: 1 y_hat 0
comment index: 1948 y_test: 1 y_hat 0
comment index: 1949 y_test: 1 y_hat 0
comment index: 1950 y_test: 1 y_hat 0
comment index: 1952 y_test: 1 y_hat 0
comment index: 1953 y_test: 1 y_hat 0
comment index: 1957 y_test: 1 y_hat 0
comment index: 1958 y_test: 1 y_hat 0
comment index: 1959 y_test: 1 y_hat 0
comment index: 1960 y_test: 1 y_hat 0
comment index: 1961 y_test: 1 y_hat 0
comment index: 1962 y_test: 1 y_hat 0
comment index: 1963 y_test: 1 y_hat 0
comment index: 1964 y_test: 1 y_hat 0
comment index: 1965 y_test: 1 y_hat 0
comment index: 1966 y_test: 1 y_hat 0
comment index: 1967 y_test: 1 y_hat 0
comment index: 1968 y_test: 1 y_hat 0
comment index: 1969 y_test: 1 y_hat 0
comment index: 1970 y_test: 1 y_hat 0
comment index: 1971 y_test: 1 y_hat 0
comment index: 1972 y_test: 1 y_hat 0
comment index: 1973 y_test: 1 y_hat 0
comment index: 1974 y_test: 1 y_hat 0
comment index: 1975 y_test: 1 y_hat 0
comment index: 1976 y_test: 1 y_hat 0
comment inde

comment index: 2250 y_test: -1 y_hat 0
comment index: 2251 y_test: -1 y_hat 0
comment index: 2252 y_test: -1 y_hat 0
comment index: 2253 y_test: -1 y_hat 0
comment index: 2254 y_test: -1 y_hat 0
comment index: 2255 y_test: -1 y_hat 0
comment index: 2256 y_test: -1 y_hat 0
comment index: 2257 y_test: -1 y_hat 0
comment index: 2258 y_test: -1 y_hat 0
comment index: 2259 y_test: -1 y_hat 0
comment index: 2260 y_test: -1 y_hat 0
comment index: 2261 y_test: -1 y_hat 0
comment index: 2262 y_test: -1 y_hat 0
comment index: 2263 y_test: -1 y_hat 0
comment index: 2264 y_test: -1 y_hat 0
comment index: 2266 y_test: -1 y_hat 0
comment index: 2267 y_test: -1 y_hat 0
comment index: 2268 y_test: -1 y_hat 0
comment index: 2269 y_test: -1 y_hat 0
comment index: 2270 y_test: -1 y_hat 0
comment index: 2271 y_test: -1 y_hat 0
comment index: 2272 y_test: -1 y_hat 0
comment index: 2273 y_test: -1 y_hat 0
comment index: 2274 y_test: -1 y_hat 0
comment index: 2275 y_test: -1 y_hat 0
comment index: 2276 y_tes